In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [2]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score
from imblearn.metrics import classification_report_imbalanced

In [3]:
# load the data
merged_df = pd.read_csv("merged_df.csv", index_col=[0],sep=',')
merged_df.head()

,GeoFips,county,pcp_count,population,pop_density,per_capita_income,pcp_per_capita,pop_density_lvl,region_far_west,region_great_lakes,...,state_TN,state_TX,state_UT,state_VA,state_VT,state_WA,state_WI,state_WV,state_WY,pcp_per_capita_bins
0,12086,Miami-Dade,13619,2715516,552.187071,57213,5.015253,3,0,0,...,0,0,0,0,0,0,0,0,0,1
1,6025,Imperial,236,180216,16.659991,44500,1.309540,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,40109,Oklahoma,2914,782051,425.989528,56971,3.726100,3,0,0,...,0,0,0,0,0,0,0,0,0,1
3,39115,Morgan,19,14702,13.631412,38583,1.292341,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,8059,Jefferson,1459,570427,288.167016,68829,2.557733,3,0,0,...,0,0,0,0,0,0,0,0,0,1


In [4]:
# Create features
X = merged_df.drop(columns=["pcp_per_capita", "pcp_per_capita_bins", "pop_density"])
X = pd.get_dummies(X) 

# Create target
# y = pcp_merged_df["pcp_per_capita"] #pcp_per_capita won't resample beacause it's continuous
y = merged_df["pcp_per_capita_bins"]

In [5]:
X.describe()

,GeoFips,pcp_count,population,per_capita_income,pop_density_lvl,region_far_west,region_great_lakes,region_mideast,region_new_england,region_plains,...,county_Yellowstone,county_Yoakum,county_Yolo,county_York,county_Young,county_Yuba,county_Yukon-Koyukuk,county_Yuma,county_Zapata,county_Zavala
count,3029.000000,3029.000000,3.029000e+03,3029.000000,3029.000000,3029.000000,3029.000000,3029.000000,3029.000000,3029.000000,...,3029.00000,3029.00000,3029.00000,3029.000000,3029.00000,3029.00000,3029.00000,3029.000000,3029.00000,3029.00000
mean,29981.758666,339.705513,1.141870e+05,48949.537801,1.496864,0.059756,0.144272,0.059426,0.022450,0.193793,...,0.00033,0.00033,0.00033,0.001321,0.00033,0.00033,0.00033,0.000660,0.00033,0.00033
std,15070.955336,1094.505146,4.231330e+05,13078.053879,1.118362,0.237073,0.351423,0.236458,0.148165,0.395334,...,0.01817,0.01817,0.01817,0.036322,0.01817,0.01817,0.01817,0.025692,0.01817,0.01817
min,1001.000000,1.000000,4.590000e+02,22644.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000
25%,18153.000000,18.000000,1.196200e+04,40971.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000
50%,29095.000000,56.000000,2.721600e+04,46571.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000
75%,44001.000000,198.000000,7.152200e+04,53791.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000
max,56045.000000,18526.000000,1.009805e+07,220645.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.00000,1.00000,1.00000,1.000000,1.00000,1.00000,1.00000,1.000000,1.00000,1.00000


In [6]:
# Check the balance of our target values
y.value_counts()

0    1519
1    1510
Name: pcp_per_capita_bins, dtype: int64

Stratify Data

In [7]:
# Train, test, split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
    y, random_state=1, stratify=y)

Change estimators from 100 to 200

In [8]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
random_forest = BalancedRandomForestClassifier(n_estimators = 200)
random_forest = random_forest.fit(X_train, y_train)

In [9]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = random_forest.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.8416040100250627

In [10]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test, y_pred)
print(matrix)

[[332  48]
 [ 72 306]]


In [11]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.82      0.87      0.81      0.85      0.84      0.71       380
          1       0.86      0.81      0.87      0.84      0.84      0.70       378

avg / total       0.84      0.84      0.84      0.84      0.84      0.71       758



In [12]:
# List the features sorted in descending order by feature importance
feature_names = X.columns
sorted(zip(random_forest.feature_importances_, feature_names), reverse=True)

[(0.21638296073433472, 'pcp_count'),
 (0.11403732899605132, 'population'),
 (0.08835486704454308, 'per_capita_income'),
 (0.07101913152365284, 'GeoFips'),
 (0.03529093493964894, 'pop_density_lvl'),
 (0.007802863662739078, 'region_southwest'),
 (0.006987337882077026, 'region_southeast'),
 (0.00599871854731415, 'state_TX'),
 (0.004794880579820735, 'region_far_west'),
 (0.0038213734744129814, 'region_plains'),
 (0.003820267652122992, 'state_KY'),
 (0.003791648055576989, 'region_new_england'),
 (0.003564568079526248, 'state_GA'),
 (0.003393044302186489, 'state_VA'),
 (0.0033170964886534314, 'region_mideast'),
 (0.003288197471554849, 'region_great_lakes'),
 (0.003198113151042924, 'state_KS'),
 (0.00315443439389689, 'state_CA'),
 (0.002905218325154723, 'county_Union'),
 (0.0028498935970968234, 'region_rocky_mountain'),
 (0.002799467826619172, 'state_MO'),
 (0.0027946423776112135, 'state_OK'),
 (0.002648664339092726, 'county_Washington'),
 (0.0024887687703486106, 'state_WI'),
 (0.002393306738

Change estimators from 100 to 200

In [13]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
easy = EasyEnsembleClassifier(n_estimators = 200,random_state=1)
easy = easy.fit(X_train, y_train)

In [14]:
# Calculated the balanced accuracy score
y_pred = easy.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9182191590086326

In [15]:
# Display the confusion matrix
y_pred = easy.predict(X_test)
matrix = confusion_matrix(y_test, y_pred)
print(matrix)

[[347  33]
 [ 29 349]]


In [16]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.92      0.91      0.92      0.92      0.92      0.84       380
          1       0.91      0.92      0.91      0.92      0.92      0.84       378

avg / total       0.92      0.92      0.92      0.92      0.92      0.84       758



In [17]:
# find r-squared score
r2_score(y_test, y_pred)

0.6728209412419939